<a href="https://colab.research.google.com/drive/1HT0hKJMFmtxksAIJWVkPt8ZvgPveRoK6?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### Chain-of-Verification (CoVe)

In [1]:
!pip install -qU google-generativeai

In [2]:
import google.generativeai as genai
import getpass

Get free-tier Google's Gemini API Key here: https://aistudio.google.com/app/apikey

In [3]:
API_KEY = getpass.getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [5]:
genai.configure(api_key=API_KEY)

In [6]:
class CoVeAgent:
    def __init__(self):
        self.model = genai.GenerativeModel("gemini-2.0-flash-exp")
        self.baseline_response = None
        self.verification_qa = []
        self.discrepancies = []

    def generate_baseline(self, query):
        """Step 1: Generate initial response"""
        prompt = f"""Answer this query:

        {query}

        Response:"""

        response = self.model.generate_content(prompt).text
        self.baseline_response = response.strip()
        return self.baseline_response

    def create_verification_questions(self, query, baseline):
        """Step 2: Generate verification questions"""
        prompt = f"""Original Query: {query}

        Initial Response:
        {baseline}

        Generate 3-5 verification questions to test the correctness and assumptions in this response.
        Each question should probe a specific claim or fact.

        Verification Questions (numbered):"""

        response = self.model.generate_content(prompt).text

        # Parse questions
        questions = []
        for line in response.split("\n"):
            line = line.strip()
            if line and (line[0].isdigit() or line.startswith("-")):
                question = line.lstrip("0123456789.-) ").strip()
                if question and len(question) > 10 and "?" in question:
                    questions.append(question)

        return questions

    def answer_verification_independently(self, question):
        """Step 3: Answer verification question independently"""
        prompt = f"""Answer this verification question independently, without referencing previous responses:

        {question}

        Provide a factual, objective answer:"""

        response = self.model.generate_content(prompt).text
        return response.strip()

    def cross_check_and_revise(self, query, baseline, verification_qa):
        """Step 4: Cross-check and revise based on verifications"""
        qa_text = "\n\n".join([
            f"Q: {q}\nA: {a}"
            for q, a in verification_qa
        ])

        prompt = f"""Original Query: {query}

        Initial Response:
        {baseline}

        Verification Q&A:
        {qa_text}

        Cross-check: Are there any discrepancies between the initial response and verification answers?
        If yes, revise the response to be more accurate. If no discrepancies, confirm the response is correct.

        Final Revised Response:"""

        response = self.model.generate_content(prompt).text
        return response.strip()

    def verify(self, query):
        """Main Chain-of-Verification pipeline"""
        print(f"\n{'='*60}")
        print(f"🔍 Chain-of-Verification (CoVe)")
        print(f"{'='*60}")
        print(f"Query: {query}\n")

        # Step 1: Generate baseline response
        print(f"{'─'*60}")
        print(f"STEP 1: Generate Baseline Response")
        print(f"{'─'*60}\n")

        baseline = self.generate_baseline(query)
        print(f"Baseline Response:\n{baseline}\n")

        # Step 2: Create verification questions
        print(f"{'─'*60}")
        print(f"STEP 2: Create Verification Questions")
        print(f"{'─'*60}\n")

        verification_questions = self.create_verification_questions(query, baseline)

        print(f"Generated {len(verification_questions)} verification questions:\n")
        for i, q in enumerate(verification_questions, 1):
            print(f"{i}. {q}")
        print()

        # Step 3: Answer each verification question independently
        print(f"{'─'*60}")
        print(f"STEP 3: Independent Verification")
        print(f"{'─'*60}\n")

        self.verification_qa = []

        for i, question in enumerate(verification_questions, 1):
            print(f"Verifying {i}/{len(verification_questions)}:")
            print(f"Q: {question}")

            answer = self.answer_verification_independently(question)
            self.verification_qa.append((question, answer))

            print(f"A: {answer[:150]}...\n")

        # Step 4: Cross-check and revise
        print(f"{'─'*60}")
        print(f"STEP 4: Cross-Check and Revise")
        print(f"{'─'*60}\n")

        final_response = self.cross_check_and_revise(query, baseline, self.verification_qa)

        print(f"{'='*60}")
        print(f"✅ FINAL VERIFIED RESPONSE")
        print(f"{'='*60}")
        print(final_response)
        print()

        return final_response

    def compare_baseline_vs_verified(self, query):
        """Compare baseline vs verified response"""
        print(f"\n{'='*70}")
        print(f"📊 COMPARISON: Baseline vs Verified")
        print(f"{'='*70}\n")

        # Generate baseline
        print("Baseline (no verification):")
        baseline = self.generate_baseline(query)
        print(f"{baseline}\n")

        # Generate verified
        print("\n" + "="*70 + "\n")
        verified = self.verify(query)

        print(f"{'='*70}")
        print(f"SUMMARY")
        print(f"{'='*70}")
        print("Baseline response generated without verification.")
        print("Verified response went through CoVe pipeline with fact-checking.")
        print()

In [ ]:
# Example 1: Hallucination Reduction
print("="*60)
print("EXAMPLE 1: Reducing Hallucinations")
print("="*60)

agent1 = CoVeAgent()
agent1.verify(
    "What are the main causes of World War I?"
)


# Example 2: Factual Accuracy
print("\n" + "="*60)
print("EXAMPLE 2: Factual Verification")
print("="*60)

agent2 = CoVeAgent()
agent2.verify(
    "How many moons does Jupiter have and when was the first one discovered?"
)


# Example 3: Scientific Information
print("\n" + "="*60)
print("EXAMPLE 3: Scientific Fact Checking")
print("="*60)

agent3 = CoVeAgent()
agent3.verify(
    "What is the speed of light and how was it first measured?"
)


# Example 4: Historical Events
print("\n" + "="*60)
print("EXAMPLE 4: Historical Accuracy")
print("="*60)

agent4 = CoVeAgent()
agent4.verify(
    "When did the Berlin Wall fall and what were the immediate consequences?"
)


# Example 5: Technical Information
print("\n" + "="*60)
print("EXAMPLE 5: Technical Verification")
print("="*60)

agent5 = CoVeAgent()
agent5.verify(
    "How does blockchain technology work and what are its main use cases?"
)


# Example 6: Comparison Test
print("\n" + "="*60)
print("EXAMPLE 6: Baseline vs Verified Comparison")
print("="*60)

agent6 = CoVeAgent()
agent6.compare_baseline_vs_verified(
    "What are the health benefits of vitamin D and what is the recommended daily intake?"
)


# Example 7: Complex Multi-Fact Query
print("\n" + "="*60)
print("EXAMPLE 7: Multi-Fact Verification")
print("="*60)

agent7 = CoVeAgent()
agent7.verify(
    "What are the three largest countries by land area and what are their populations?"
)


# Example 8: Knowledge-Intensive Question
print("\n" + "="*60)
print("EXAMPLE 8: Knowledge-Intensive Tutoring")
print("="*60)

agent8 = CoVeAgent()
agent8.verify(
    "Explain photosynthesis: what are the inputs, outputs, and main stages?"
)


print("✅ Chain-of-Verification Complete!")

EXAMPLE 1: Reducing Hallucinations

🔍 Chain-of-Verification (CoVe)
Query: What are the main causes of World War I?

────────────────────────────────────────────────────────────
STEP 1: Generate Baseline Response
────────────────────────────────────────────────────────────

Baseline Response:
The main causes of World War I are often summarized using the acronym **MAIN**:

*   **Militarism:** The glorification of military power and the belief that a nation should maintain a strong military to aggressively promote its interests. This led to an arms race, particularly between Germany and Great Britain, creating a climate of fear and suspicion.
*   **Alliances:** A complex network of treaties and agreements that obligated nations to defend each other in case of attack. These alliances, such as the Triple Alliance (Germany, Austria-Hungary, Italy) and the Triple Entente (France, Great Britain, Russia), meant that a localized conflict could quickly escalate into a larger war.
*   **Imperialis